Com o Resultado do `fpgrowth`, descobrimos que a carta `jirachi` é uma carta amplamante utilizadas entre os jogadores nos torneios de 2019.
Iremos utilizar um algoritmo SD como alvo o ranking dos jogadores variando de 0 para ranks piores que `RANK_ANALISAR` e 1 o contrario, nos torneios do ano de 2019 para encontrar informacoes do que precisamos para criar um deck ideal e ver se o algoritmo recomendara a carta `jirachi`.

## Variaveis para definir

In [89]:
CONSIDERAR_TREINADORES = True
ANO_ANALISAR = 2019
RANK_ANALISAR = 5

# quantidade de cartas de 1 tipo que deve exitir no dataset para ser considerado evitando outliers 
CARTAS_MINIMAS = 10

## Bibliotecas utilizadas e carregando dataset

In [90]:
import numpy as np
import pandas as pd
import pysubgroup as ps

df_original = pd.read_csv('./Data/tournaments.csv')
df = df_original.copy()


## Eliminar cartas de treinador se quiser

In [91]:
if not CONSIDERAR_TREINADORES:
    df = df[df['type_card'] != 'Trainer']

## Filtragem de colunas no dataset e limpeza de linhas com coluna regiao do torneio vazias

In [92]:
columns_to_remove = [
    'id_card', 'amount_card', 'price_card', 'combo_type_id', 'name_tournament',
    'combo_type_name', 'name_player', 'category_tournament', 'type_card',
    'month_tournament', 'day_tournament', 'valid_rotation_at_tournament', 
    'rotation_name', 'year_begin', 'all_time_score', 'country_tournament', 'country_player',
    'month_begin', 'day_begin',
]
df.drop(columns=columns_to_remove, inplace=True)
df = df[df['region_tournament'].notna()]

df.head(1)

,name_card,energy_type_card,id_player,ranking_player_tournament,id_tournament,region_tournament,year_tournament
7462,Charmander,Fire,649,1,385,SA,2023


## Adaptando dados para pordermos usar SD

Como somente `ranking_player` do tipo numerico, vamos transformalo em binario em que menor que `RANK_ANALISAR` seria 1, estamos buscando cartas que aumentem a probabilidade de um jogador subir de rank acima de `RANK_ANALISAR`

In [93]:
# todos os ranking players acima de 15 sao 1
df['ranking_player_tournament'] = df['ranking_player_tournament'].apply(lambda x: 1 if x < RANK_ANALISAR else 0)

df.head(1)

,name_card,energy_type_card,id_player,ranking_player_tournament,id_tournament,region_tournament,year_tournament
7462,Charmander,Fire,649,1,385,SA,2023


## Separando por regiao e verificando a qualidade dos subgrupos de cartas que aumentem a probabilidade de um jogador subir no rank

In [94]:
def elimina_cards_menos_usados(df):
    cartas = df['name_card'].unique()
    for carta in cartas:
        if len(df[df['name_card'] == carta]) < CARTAS_MINIMAS:
            df = df[df['name_card'] != carta]
    return df

def agrupa_deck_rank(df_region):
    #filtra torneios da regiao
    torneios = df_region['id_tournament'].unique()

    lista_decks_rank = []
    for torneio in torneios:
        df_torneio = df_region[df_region['id_tournament'] == torneio]
        jogadores = df_torneio['id_player'].unique()
        for jogador in jogadores:
            df_jogador = df_torneio[df_torneio['id_player'] == jogador]
            cartas_jogador_rank = df_jogador['name_card'].unique().tolist()
            cartas_jogador_rank.append(df_jogador['ranking_player_tournament'].iloc[0])
            lista_decks_rank.append(cartas_jogador_rank)
    return lista_decks_rank

def subgrupo_dataset_region(df_region):
    lista_decks_rank = agrupa_deck_rank(df_region)
    cartas = df_region['name_card'].unique()
    data = []
    for deck_rank in lista_decks_rank:
        pokemons_no_deck_rank = {carta: (carta in deck_rank[:len(deck_rank)-1]) for carta in cartas}
        pokemons_no_deck_rank['ranking_player_tournament'] = deck_rank[-1]
        data.append(pokemons_no_deck_rank)
    # Converte a lista de dicionários em um DataFrame
    df_decks_cartas = pd.DataFrame(data)

    return df_decks_cartas

def calcula_subgrupo(df):
    target = ps.BinaryTarget('ranking_player_tournament',1)
    search_space = ps.create_selectors(df, ignore=['ranking_player_tournament'])
    task = ps.SubgroupDiscoveryTask(
        df,
        target, 
        search_space, 
        result_set_size=20, 
        depth=5, 
        qf=ps.WRAccQF()  # Quality function to evaluate subgroups
    )
    result = ps.BeamSearch().execute(task)
    return result

### Por regiao

In [95]:
regions = df['region_tournament'].unique()
for region in regions:
    print(f"Região: {region}\n")
    df_region = df[(df['region_tournament'] == region) & (df['year_tournament'] == ANO_ANALISAR)]

    # verifica se tem torneios na regiao
    if len(df_region) == 0:
        print(f'Sem torneios na região {region} no ano {ANO_ANALISAR}\n')
        print("-------------------------------------------"*3)
        continue

    # calcula subgrupo
    df_region = elimina_cards_menos_usados(df_region)
    dataset = subgrupo_dataset_region(df_region)
    result = calcula_subgrupo(dataset)

    # imprime resultados
    for row in result.to_dataframe().itertuples():
        print(f"Subgrupo: {row.subgroup}")
    print()
    print(result.to_dataframe(), "\n")
    print("-------------------------------------------"*3)

Região: SA

Subgrupo: Acro Bike==False AND Cryogonal==False AND Custom Catcher==False AND Mew==False AND Professor Elm's Lecture==False
Subgrupo: Acro Bike==False AND Cryogonal==False AND Custom Catcher==False AND Mew==False AND Pidgey==False
Subgrupo: Cryogonal==False AND Custom Catcher==False AND Latios-GX==False AND Mew==False AND Professor Elm's Lecture==False
Subgrupo: Cryogonal==False AND Custom Catcher==False AND Latios-GX==False AND Mew==False AND Pidgey==False
Subgrupo: Acro Bike==False AND Cryogonal==False AND Custom Catcher==False AND Latios-GX==False AND Professor Elm's Lecture==False
Subgrupo: Acro Bike==False AND Cryogonal==False AND Custom Catcher==False AND Latios-GX==False AND Pidgey==False
Subgrupo: Acro Bike==False AND Cryogonal==False AND Custom Catcher==False AND Latios-GX==False AND Mew==False
Subgrupo: Acro Bike==False AND Cryogonal==False AND Custom Catcher==False AND Latios-GX==False AND Mallow & Lana==True
Subgrupo: Acro Bike==False AND Cryogonal==False AND Cu

### Ignorando Regioes

In [96]:
df_all = df[(df['year_tournament'] == ANO_ANALISAR)]

# verifica se tem torneios
if len(df_all) == 0:
    print(f'Sem torneios no ano {ANO_ANALISAR}')
    print("-------------------------------------------"*3)
    exit()

# calcula subgrupo
df_all = elimina_cards_menos_usados(df_all)
dataset = subgrupo_dataset_region(df_all)
result = calcula_subgrupo(dataset)

# imprime resultados
for row in result.to_dataframe().itertuples():
    print(f"Subgrupo: {row.subgroup}")
print()
print(result.to_dataframe(), "\n")
print("-------------------------------------------"*3)

Subgrupo: Acro Bike==False AND Cryogonal==False AND Energy Spinner==False AND Rescue Stretcher==False AND Switch==True
Subgrupo: Acro Bike==False AND Energy Spinner==False AND Malamar==False AND Rescue Stretcher==False AND Switch==True
Subgrupo: Acro Bike==False AND Energy Spinner==False AND Inkay==False AND Rescue Stretcher==False AND Switch==True
Subgrupo: Acro Bike==False AND Energy Spinner==False AND Keldeo-GX==False AND Rescue Stretcher==False AND Switch==True
Subgrupo: Acro Bike==False AND Ditto ♢==False AND Energy Spinner==False AND Rescue Stretcher==False AND Switch==True
Subgrupo: Acro Bike==False AND Energy Spinner==False AND Phione==False AND Rescue Stretcher==False AND Switch==True
Subgrupo: Acro Bike==False AND Energy Spinner==False AND Rescue Stretcher==False AND Switch==True AND Tate & Liza==False
Subgrupo: Acro Bike==False AND Energy Spinner==False AND Giratina==False AND Rescue Stretcher==False AND Switch==True
Subgrupo: Acro Bike==False AND Energy Spinner==False AND G